## This notebook shows how to produce gif from Kr maps and corresponding histograms (outputs of automatic mat production script)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML, Image
import datetime
from glob import glob
import seaborn as sns
import re
from invisible_cities.reco.corrections_new import read_maps

In [2]:
rc('animation', html='html5')

In [17]:
%%capture
! wget https://www.dropbox.com/s/mv0za2rkwfgf3p1/hist_7550_config_LBphys.h5
! wget https://www.dropbox.com/s/47006fh5jiv9qwe/hist_7567_config_LBphys.h5
! wget https://www.dropbox.com/s/6cblspe9itrqega/hist_7568_config_LBphys.h5
! wget https://www.dropbox.com/s/4urvmrvktz9l1vw/hist_7569_config_LBphys.h5
! wget https://www.dropbox.com/s/o7ung2mu23dqtmj/hist_7570_config_LBphys.h5
! wget https://www.dropbox.com/s/rkev3vy9x8z4nb0/map_7550.h5
! wget https://www.dropbox.com/s/a08l83wzac7qj1h/map_7567.h5
! wget https://www.dropbox.com/s/a3lv93vqn54zeux/map_7568.h5
! wget https://www.dropbox.com/s/46nvcipb9s1kj2t/map_7569.h5
! wget https://www.dropbox.com/s/q1r7grog8qiybhw/map_7570.h5


Here the maps_list is sorted and then we look for histogram file with the same run number to create hist_list

In [7]:
maps_list  = sorted(glob('map_????.h5'))
hist_list = []
for mapfname in maps_list:
    hist_list.extend(glob('hist_{}*.h5'.format(re.findall(r'\d+',mapfname)[0])))
    
assert len(maps_list) == len(hist_list)

The interesting one might be kr rate after all selections, ill use that one for now. Lets loop over hist files, create one big dataframe to plot

In [8]:
hists_dfs = []
for hist_file in hist_list:
    hist_df   = pd.read_hdf(hist_file, key='rate_after_sel')
    to_Hz_fac =  hist_df.magnitude.unique()[1]-hist_df.magnitude.unique()[0]
    hist_df.entries /= to_Hz_fac
    hist_df = hist_df.assign(datetime = lambda df : df.magnitude.apply(datetime.datetime.fromtimestamp))
    hists_dfs.append(hist_df)
hists_merged = pd.concat(hists_dfs)

Here we set the figure where the animation will happen

In [28]:
%%capture
fig = plt.figure(figsize = (10, 10),frameon = False)
gs = fig.add_gridspec(2, 40)
ax1 = fig.add_subplot(gs[0, 0:16])
ax1cb = fig.add_subplot(gs[0, 16])
ax2 = fig.add_subplot(gs[0, 20:36])
ax2cb = fig.add_subplot(gs[0, 36])
ax3 = fig.add_subplot(gs[1, :])


init and animate functions use already defined figure and its axes

In [29]:
def init():
    mapi = read_maps(maps_list[0])
    maprun = mapi.mapinfo.run_number
    fig.suptitle('Run {}'.format(maprun), fontsize=20)
    sns.heatmap(mapi.e0, square=True, ax = ax1, xticklabels=False, yticklabels=False, cbar_ax = ax1cb, vmin=7000, vmax=14000)
    ax1.set_title('E0')
    ax1cb.set_title('pes')
    sns.heatmap(mapi.lt, square=True, ax = ax2, xticklabels=False, yticklabels=False, cbar_ax = ax2cb, vmin=4000, vmax=16000)
    ax2.set_title('LT')
    ax2cb.set_title(r'$\mu$s')
    ax3.step(hists_merged.datetime, hists_merged.entries, where='pre', color='blue');
    ax3.set_title('Kr rate')
    ax3.set_xlabel('date')
    ax3.set_ylabel('Rate (Hz)')
    ax3.tick_params(axis='x', rotation=45)
    return (ax1, ax2, ax3)

In [30]:
def animate(i):
    mapi = read_maps(maps_list[i])
    maprun = mapi.mapinfo.run_number
    ax3.clear(); ax1.clear(); ax2.clear()
    ax1.set_title('E0')
    ax2.set_title('LT')
    fig.suptitle('Run {}'.format(maprun), fontsize=20)
    sns.heatmap(mapi.e0, square=True, ax = ax1, xticklabels=False, yticklabels = False, cbar_ax = ax1cb, vmin=7000, vmax=14000)
    sns.heatmap(mapi.lt, square=True, ax = ax2, xticklabels=False, yticklabels = False,  cbar_ax = ax2cb, vmin=4000, vmax=16000)
    ax1cb.set_title('pes')
    ax2cb.set_title(r'$\mu$s')

    time_min, time_max = hists_dfs[i].datetime.min(), hists_dfs[i].datetime.max()
    ax3.clear()
    ax3.step(hists_merged.datetime, hists_merged.entries, where='pre', color='blue');
    ax3.axvspan(time_min, time_max, alpha=0.5, color='blue', edgecolor=None)
    ax3.set_title('Kr rate')
    ax3.set_xlabel('date')
    ax3.set_ylabel('Rate (Hz)')
    ax3.tick_params(axis='x', rotation=45)
    return ax1, ax2, ax3

In [31]:

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(maps_list), interval=1000, blit=False)

In [32]:
anim

<font size="4">To save gif matplotlib needs imagemagick. Can be installed to conda environment with </font>

<font size="4"> **conda install -c conda-forge imagemagick** </font>


In [22]:
anim.save('animation_map.gif', writer='imagemagick', fps=0.5)


In [23]:
Image(url='animation_map.gif')
